In [1]:
import autogen
autogen.__version__

'0.7.5'

In [5]:
#!pip install autogen-agentchat[gemini]~=0.2
!pip install ag2[gemini]
#!pip install chromadb==0.5.1
#!pip install google-generativeai == 0.8.4
#!pip install vertexai == 1.71.1
#ag2 ==0.7.3
#autogen-agentchat ==  0.4.3
#python  3.11.11


   ---------------------------------------- 0.0/7.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.3 MB 2.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.3/7.3 MB 2.3 MB/s eta 0:00:03
   -------- ------------------------------- 1.6/7.3 MB 2.0 MB/s eta 0:00:03
   ----------- ---------------------------- 2.1/7.3 MB 2.1 MB/s eta 0:00:03
   -------------- ------------------------- 2.6/7.3 MB 2.2 MB/s eta 0:00:03
   ------------------ --------------------- 3.4/7.3 MB 2.3 MB/s eta 0:00:02
   --------------------- ------------------ 3.9/7.3 MB 2.3 MB/s eta 0:00:02
   ------------------------ --------------- 4.5/7.3 MB 2.4 MB/s eta 0:00:02
   --------------------------- ------------ 5.0/7.3 MB 2.4 MB/s eta 0:00:01
   ---------------------------- ----------- 5.2/7.3 MB 2.3 MB/s eta 0:00:01
   ------------------------------- -------- 5.8/7.3 MB 2.3 MB/s eta 0:00:01
   -----------------------

In [3]:
#import agentops
#agentops.init("954ecbcb-9555-470b-a2ad-a6999473648b")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=d3aef696-c56b-4437-9931-f8339903f90f


In [2]:
#Path helps in handling file system paths in an object-oriented way.
from pathlib import Path
#ConversableAgent is used to create an AI agent capable of executing code or conversing.
from autogen import ConversableAgent
#CodeBlock: A structure for handling and running code blocks.
#LocalCommandLineCodeExecutor: A local environment for executing code in a command-line interface.
from autogen.coding import  CodeBlock, LocalCommandLineCodeExecutor
#Creates a directory named "coding" using Path("coding").
#mkdir(exist_ok=True) ensures that the directory is created only if it doesn't already exist.
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
#Creates an instance of LocalCommandLineCodeExecutor, which allows executing code in a local command-line environment.
#The work_dir parameter specifies that all code execution will happen inside the "coding" directory.
executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

#Agent 1
#Creates an instance of ConversableAgent, naming it "code_executor_agent".
#This agent will be responsible for handling code execution.
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,#This means the agent will not rely on an AI model to process queries—it will only handle code execution.
    code_execution_config={
        "executor": executor,#This enables the agent to execute code locally in the "coding" directory.
    },
    human_input_mode="NEVER",#Disables human input, meaning the agent will automatically execute code without asking for manual confirmation.
)

In [5]:
# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, 
    print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the 
    task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your 
language skill.When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other 
action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. 
Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

import os

#Agent 2

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "gemini-1.5-flash", "api_key":"","api_type": "google"}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",)

In [6]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message=" write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count."
)

pprint.pprint(chat_result)

code_executor_agent (to code_writer):

 write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

```python
# filename: permutation_count.py
from collections import Counter

def count_permutations(word):
    """Counts the number of permutations of a word, accounting for repeated letters."""
    char_counts = Counter(word.lower())
    factorial_n = 1
    for i in range(1, len(word) + 1):
        factorial_n *= i

    denominator = 1
    for count in char_counts.values():
        for i in range(1, count + 1):
            denominator *= i
    return factorial_n // denominator

word = "ALGEbRA"
result = count_permutations(word)
print(f"The number of permutations for the word '{word}' is: {result}")

```


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BL

In [7]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message="""Write optimized Python code to calculate the number of unique permutations of the word 'ALGEBRA.'
    For example, the word 'ALGEbRA' should have 2520 unique permutations. Use efficient logic to compute the result and verify the output using 
    the word 'ALGEBRA.' The code should only return the final permutation count.""")
pprint.pprint(chat_result)
#agentops.end_session("Success")

code_executor_agent (to code_writer):

Write optimized Python code to calculate the number of unique permutations of the word 'ALGEBRA.'
    For example, the word 'ALGEbRA' should have 2520 unique permutations. Use efficient logic to compute the result and verify the output using 
    the word 'ALGEBRA.' The code should only return the final permutation count.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

```python
# filename: permutation_counter.py
import math

def count_unique_permutations(word):
    """
    Calculates the number of unique permutations of a given word.

    Args:
        word: The input word (string).

    Returns:
        The number of unique permutations (integer).  Returns 0 if the input is invalid.
    """
    if not isinstance(word, str) or not word:
        return 0

    char_counts = {}
    for char in word.lower():
        char_counts[char] = char_counts.get(char, 0) + 1

    n = len(w

In [8]:
dir(chat_result)


['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chat_history',
 'chat_id',
 'cost',
 'human_input',
 'summary']

In [9]:
chat_result.human_input

[]

In [10]:
chat_result.chat_id

In [11]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.000356925,
  'gemini-1.5-flash': {'cost': 0.000356925,
   'prompt_tokens': 2259,
   'completion_tokens': 625,
   'total_tokens': 2884}},
 'usage_excluding_cached_inference': {'total_cost': 0.000356925,
  'gemini-1.5-flash': {'cost': 0.000356925,
   'prompt_tokens': 2259,
   'completion_tokens': 625,
   'total_tokens': 2884}}}

In [11]:
chat_result.human_input

[]

In [12]:
chat_result.summary

''